## Reinforcement Learning

* Введение
    * Обучение с учителем
    * Обучение без учителя
    * Обучение с подкреплением

* RL - терминология и отличие этого метода обучения
    * Agent - Агент
    * State - Состояние среды
    * Reward - Функция награды
    
* Особенности RL, cложности
    * Низкая скорость обучения
    * Сложное проектирование функции награды
    * Невоспроизводимость обучения
    
* Области применения
    * Расширить раздел
    
* Сходство с Марковским процессом
    * 

* Gym - среда для обучения агентов
    * Описание
    * Пример работы
    
    
* Обучение сетей DQN
    * Q-learning
    * DQN
    * Experience Replay Buffer
    * TD-Loss
    * Другие улучшения DQN
    
* Заключение

* Ссылки

## Введение

### Обучение с учителем

Если у нас есть набор примеров с правильными ответами, то мы используем эту выборку для обучения нашей модели, а после обучения, применяем её к неразмеченным данным. Именно этот подход мы использовали, когда обучали классификатор для MNIST, подавая на вход сети картинки с изображениями рукописных цифр и считая градиент для подстройки весов на основе разницы между известным лэйблом цифры и выходом нейросети.

### Обучение без учителя

В некоторых случаях у нас нет размеченных данных, на которых мы могли бы заранее обучить модель. Но, при решении некоторых задач, можно обойтись без размеченной выборки. Примером такой задачи является задача кластеризации.

### Обучение с подкреплением

В некоторых случаях существующие методы обучения без учителя нам не подходят. В то же время у нас нет возможности создать качественную обучающую выборку. При этом мы можем постфактум оценить действия нашей модели и использовать эту оценку подстроить модель так, чтобы она чаще совершала желательные действия и реже - нежелательные. В литературе такую оценку называют вознаграждением (reward), а обучение строится таким образом, чтобы это модель стремилась максимизировать получаемое вознаграждение.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/zap/agend_and_environment.gif" width="600"/>

- Неопределенность: действия и награды случайны
- Отложенность r_t+1 может не зависеть от a_t 

## RL - терминология и отличие этого метода обучения
    
### Агент

Агент - это программа, принимающая решение о дальнейших действиях на основе информации о состоянии среды. В нашем случае агентом будет нейронная сеть.

### Среда

Среда - это мир, в котором агент должен "выживать", т.е. всё, с чем агент может прямо или косвенно взаимодействовать. Среда обладает состоянием (**state**), агент может влиять на среду, совершая какие-то действия (**Actions**), переводя среду при этом из одного состояния в другое и получая какое-то вознаграждение. Среда описывается пространством возможных состояний. Конкретное состояние - вектор в этом пространстве.

В зависимости от конкретной задачи, агент может наблюдать либо полное состояние среды, либо только некоторую его часть. Во втором случае, агенту может потребоваться какое-то внутреннее представление полного состояния, которое будет обновляться по мере получения новых данных.

### Функция награды

Фукнция награды (**Reward**) - вводимая программистом формула вычисления ценности действия на основе финального результата, ожидания этого результата, промежуточных результатов и любых других параметров, которые будут подсказывать путь к наилучшей последовательности действий агента. Это некоторый аналог функции потерь, без которой непонятно чему учиться. Например, в шахматах истинная награда это победа, но взятая фигура соперника тоже ценна и должна увеличивать награду, если мы хотим подсказать агенту, что брать чужие фигуры полезно. Может ли после этого агент получать мат, позарившись на незащищенную фигуру? Да, ровно как и неопытный шахматный игрок. Попытка передать через дополнительные неосновные награды подсказки к получению основной награды называется **reward shaping**.

## Особенности RL. Сложности

### Низкая скорость обучения (sample efficiency)

Общая проблема всех алгоритмов обучения с подкреплением - низкая скорость обучения. В то время, как человеку может быть достаточно одного повторения, чтобы выучить какое-то действие, агенту RL требуется десятки тысяч повторений даже в простых задачах. В какой-то степени это связано с несовершенством архитектуры, но самый большой вклад даёт тот факт, что человек может использовать накопленный в прошлом опыт из других областей. Игра Montezuma's Revenge - популярный подопытная среда для RL в последнее время. И яркий пример низкой эффективности повторений  у алгоритмов RL по сравнению с человеком. 

Челокек, как правило, быстро понимает, что нужно избегать черепа и забрать ключ, гравитация направлена вниз, а падение с большой высоты опасно. Алгоритму же приходиться обучаться с полного нуля. Если же подменить элементы интерфейса на неочевидные для человека, то его sample-efficency тоже сильно падает (хотя всё-равно лучше, чем RL).

<img src="https://edunet.kea.su/repo/src/L15_RL/img/zap/game_prior.gif" width="400">

А теперь то же самое, но в нечеловекочитаемом виде. Для RL разницы нет, а для человека сразу стало сложнее.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/zap/game_no_prior.gif" width="400">

### Сложное проектирование функции награды
Так же важным фактором являются редкие награды. Часто в ходе одного эпизода алгоритм делает множество различных действий, а награду получает только в конце. Соответственно, веса сети можно обновить только в конце эпизода и нельзя поощрить или наказать конкретные действия внутри эпизода. В итоге требуется большое количество повторений для достижения оптимальных весов.

Один из способов улучшить эффективность при редких наградах - reward shaping - модификация функции награды так, чтобы явно поощрялись какие-то действия внутри эпизода. Но качественно сконструировать такую функцию тяжело, а ошибки в ней могут приводить к неожиданным эффектам.

**Пример 1** - В гонке лодок агент получал вознаграждене не только за победу в гонке, но и за сбор игровых бонусов. В итоге он решил, что гонка не очень-то и нужна, достаточно собирать бонусы.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/zap/coastrunner.gif" width="400">

**Пример 2** - У данного агента мы наблюдаем поподание в локальный минимум. Этот агент получает поощрение за набранную скорость. На начальном этапе во время случайного поиска агент обнаружил, что кувыркнуться вперёд даёт хорошее вознаграждение в начале. Постепенно, после нескольких попыток, переворачивание на спину закрепилось, как успешная стратегия. После закрепления такого поведения агент не смог выйти из этого состояния, т.к. оказалось проще научиться двигаться в таком состоянии, чем научиться переворачиваться обратно на ноги. Похожее поведение можно случано получить, если поощрять агента за то, что его ноги оторваны от земли.

<center><video controls src="https://edunet.kea.su/repo/src/L15_RL/img/zap/upsidedown_half_cheetah.mp4" width="400"> </video></center>

### Невоспроизводимость обучения
Обычно мы всегда начинаем со случайного распределения весов, что не мешает стабильно получать результат обучения. Но с RL это не так. Даже в простой задаче с 3 степенями свободы (state - трёхмерный вектор) и одной степенью воздействия (action - скаляр), обучение с разными зародышами генератора псевдослучайных чисел привело в 30% случаев к фиаско. Заранее понять, что обучение пошло плохо, нельзя или сложно.
<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/pendulum_results.png" width="400">
<center><em> episode_reward/test <em><center>

## Области применения

* Управление роботами, БПЛ
* Игры
* Принятие решений в условтях неопределенности, управление инвестициями
* drug discovery: обычно функции отражающие полезность лекарства недифференицруемы и можно искать оптимальные, вводя RL
* Не так давно одна российская металлургическая компания использовала RL для оптимаизации работы прокатного стана
* NAS (Neural Architecture Search): при поиске оптимальных топологий сети можно решать задачу с помощью RL, награждая нашего агента за нахождение ххороших топологий и наказывая за плохие

## Марковский процесс принятия решений (MDP)

Прежде чем перейти к нейронным сетям, рассмотрим марковский процесс принятия решений, который поможет нам разобраться.

**Markov property** - Вероятность перехода в новое состояние зависит только от текущего состояния и текущего действия. Это свойство позволяет применять рекурсивные алгоритмы при обходе графов.

**Марковский процесс принятия решений** (англ. Markov decision process (MDP)) — спецификация задачи последовательного принятия решений для полностью наблюдаемой среды с марковской моделью перехода и дополнительными вознаграждениями. Назван в честь Андрея Маркова, служит математической основой для того, чтобы смоделировать принятие решения в ситуациях, где результаты частично случайны и частично под контролем лица, принимающего решения. Сегодня эта спецификация используется во множестве областей, включая робототехнику, автоматизированное управление, экономику и производство.

## Пример 

Составим граф:
- вершина - состояние среды
- ребро - переход между состояниями ~= действие

Кроме того:
- Каждые переход сопровождается наградой (reward)
- Переходы случайны
- Существуют терминальные состояния

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Markov_Decision_Process.svg" width="400px" />


- черные беззнаковые числа - вероятности переходов
- знаковые это награды

Воспользуемся примером:
https://github.com/yandexdataschool/Practical_RL

In [ ]:
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week02_value_based/mdp.py
!touch .setup_complete

# After this setup we can import mdp package

In [ ]:
from mdp import MDP 

transition_probs = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's2': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}
rewards = {
    's1': {'a0': {'s0': +5}},
    's2': {'a1': {'s0': -1}}
}


mdp = MDP(transition_probs, rewards, initial_state='s0')


print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('a1')
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

In [ ]:
# MDP methods
print("mdp.get_all_states =", mdp.get_all_states())
print("mdp.get_possible_actions('s1') = ", mdp.get_possible_actions('s1'))
print("mdp.get_next_states('s1', 'a0') = ", mdp.get_next_states('s1', 'a0'))

# state, action, next_state
print("mdp.get_reward('s1', 'a0', 's0') = ", mdp.get_reward('s1', 'a0', 's0'))

# get_transition_prob(self, state, action, next_state)
print("mdp.get_transition_prob('s1', 'a0', 's0') = ", mdp.get_transition_prob('s1', 'a0', 's0'))

In [ ]:
from IPython.display import display
import mdp as mdp_package
display(mdp_package.plot_graph(mdp))

### Награда в MDP

* Каждый переход ассоциируется с наградой R.
* Награда может быть как нулевой, так положительной или отрицательной.

* Например в MDP описывающем игру в шахматы или GO награда агент может положительной только при переходе в состояние "партия выигранна". На всех остальных ходах награда будет нулевой. Это правильный подход, потому что конечная цель игры - победа. 

* Для перемещающегося робота награда на каждом шаге может быть отрицательной, так как на перемещение требуется энергия. и.т.п. 


### Дисконтирование (discounting)


Что лучше: "Получить миллион долларов через 100 лет или $100 сейчас"

Часто время получения награды имеет значение. Поэтому при оценке куммулятивной награды на шаге t (Gt) используется коэффициент дисконтирования $\gamma$ 



<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-12_.png" alt="Drawing" width="700px;"/>




<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-15.png" alt="Drawing" width="700px;"/>

### Формальное описание

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-24__.png"  width="700px;"/>



Алгоритм:

<img src="https://edunet.kea.su/repo/src/L15_RL/img/gan/mdp.png"  width="600px;"/>

## Постановка задачи

Нужно найти последовательность переходов которой будет соответствовать максимальная награда (Gt).



### Политика

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-26_.png" alt="Drawing" width="700">

Результатом ее решения будет "Политика" вероятность с которой нужно выбрать действие a из состояния s.




<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/policy1.png"  width="800px;"/>


*Стоит отметить, что политика, как следует из определения не обязательно должна быть жестко зафиксированной. Вполне вохможно, что она будет случайной. Это может, к примеру, пригодиться при исследованиях среды.*



<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/policy2.png"  width="500px;"/>

### Value - function

Для оценки политики вводятся функции V и Q 



V(s) - какое максимальное вознаграждение можно получить если начать двигаться из состояния s.

* В течении всего оставшегося времени

Q(s,a) -  анлогично, но при условии что в состоянии s (s0 = s) было выбранно действие a




<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-28_.png" alt="Drawing" width="700px;"/>

Под $E_\pi$ мы подразумеваем стохастичность и политки и среды





<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/silver-2-29_.png" alt="Drawing" width="450px;"/>


В вершинах значения V для политики $v_\pi$ (ерху картинки)


### Решение

Задача оптимальной политики решается через поиск оптимальных V* и Q* - функции.


<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-37_.png" alt="Drawing" width="700px;" />

### Bellman equation

Для ее решения используется рекуррентное уравнение [Белмана](https://en.wikipedia.org/wiki/Richard_E._Bellman)

<img src="https://edunet.kea.su/repo/src/L15_RL/img/gan/bellman.png" width="700px;"/> 

Q функция убирает стохастичность политики на первом шаге. С помощью нее мы фиксируем наш первый шаг из этого состояния.


На практике мы будем считать Q и V так:


$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$

Для получения V будем использовать Q 

$$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = \max_a Q_i(s,a)$$


<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/silver-2-39_.png" alt="Drawing" width="800px;"/>

<center><em> Example: Optimal Action-Value Function for Student MDP <em><center> 

фактически q задает политику

# Варианты решений

- Bellman Optimality Equation
- Solving the Bellman Optimality Equation
- Bellman Optimality Equation is non-linear No closed form solution (in general) Many iterative solution methods
    - Value Iteration 

    - Policy Iteration 

    - Q-learning 

    - Sarsa


- Принцип Беллмана - на каждом шаге следует выбирать оптимальное решение х в предположении об оптимальности всех последующих шагов.
- Оптимальный выбор действия зависит только от текущего состояния и не зависит от предыстории.



### Value Iteration


Алгоритм

1. Инициализация

- Создаем массив V с количеством элементов равным количеству состояний.

- Заполняем его нулями


2. Оценка политики (Policy evaluation)

- Для всех состояний считаем Q(s,a)

- Обновляем массив V[s] -> max(Q(s,a))


*В отличие от Policy iteration сама политика в памяти не храниться, она генеируется при помощи Q-функции.


3. Обновление политики (Policy improvement)

- Для каждого состояния считаем Q(s,a) для всех a

- Выбираем a для которого Q(s,a) - максимально

- Если политика изменилась, переходим к шагу 2, иначе останавливаемся


Now let's build something to solve this MDP. The simplest algorithm so far is __V__alue __I__teration

Here's the pseudo-code for VI:

---

`1.` Initialize $V^{(0)}(s)=0$, for all $s$

`2.` For $i=0, 1, 2, \dots$
 
`3.` $ \quad V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$, for all $s$

---

r(s,a,s') - награда соответствующая выбору действия a в s
s - исходное состояние 
s' - новое состояние

First, let's write a function to compute the state-action value function $Q^{\pi}$, defined as follows

$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$

s' - зависит от вероятности перехода


In [ ]:
def get_action_value(mdp, state_values, state, action, gamma):
    """ 
    Computes Q(s,a) as in formula above 
    mdp : MDP object
    state_values : dictionayry of { state_i : V_i }
    state: string id of current state
    gamma: float discount coeff
    """

    next_states = mdp.get_next_states(state, action)

    Q = 0.0

    for next_state in next_states.keys():
        p = next_states[next_state] # alternatively p = mdp.get_transition_prob(state, action, next_state)
        Q += p * (mdp.get_reward(state, action, next_state) + gamma * state_values[next_state])
    return Q 

Using $Q(s,a)$ we can now define the "next" V(s) for value iteration.
 $$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = \max_a Q_i(s,a)$$

In [ ]:
def get_new_state_value(mdp, state_values, state, gamma):
    """ Computes next V(s) as in formula above. Please do not change state_values in process. """
    if mdp.is_terminal(state):
        return 0 # Game over

    q_max = float('-inf')
    actions = mdp.get_possible_actions(state)
    for a in actions:
        q = get_action_value(mdp, state_values, state, a, gamma)
        q_max = max(q_max, q)
    return q_max

Finally, let's combine everything we wrote into a working value iteration algo.

In [ ]:
import mdp as MDP
from IPython.display import display


# parameters
gamma = 0.9            # discount for MDP
num_iter = 100         # maximum iterations, excluding initialization
# stop VI if new values are this close to old values (or closer)
min_difference = 0.001

# initialize V(s)
state_values = {s: 0 for s in mdp.get_all_states()}

display(MDP.plot_graph_with_state_values(mdp, state_values))

for i in range(num_iter):

    # Compute new state values using the functions you defined above.
    # It must be a dict {state : float V_new(state)}

    new_state_values = {}
    for s in state_values.keys():
        new_state_values[s] = get_new_state_value(mdp,state_values,s,gamma)   


    # Compute difference
    diff = max(abs(new_state_values[s] - state_values[s])
               for s in mdp.get_all_states())
    print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
    print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))
    state_values = new_state_values

    if diff < min_difference:
        print("Terminated")
        break

Здесь нет никакого "текущего состояния"!

In [ ]:
display(MDP.plot_graph_with_state_values(mdp, state_values))

Now let's use those $V^{*}(s)$ to find optimal actions in each state

 $$\pi^*(s) = argmax_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = argmax_a Q_i(s,a)$$
 
The only difference vs V(s) is that here we take not max but argmax: find action such with maximum Q(s,a).

In [ ]:
def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None
    
    best_action = None
    q_max = float('-inf')
    actions = mdp.get_possible_actions(state)
    for a in actions:
        q = get_action_value(mdp, state_values, state, a, gamma)
        if q > q_max:
            best_action = a
            q_max = q
            
    return best_action

In [ ]:
display(MDP.plot_graph_optimal_strategy_and_state_values(mdp, state_values, get_action_value))

In [ ]:
# Measure agent's average reward

s = mdp.reset()
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
    rewards.append(r)

print("average reward: ", np.mean(rewards))

assert(0.40 < np.mean(rewards) < 0.55)

# Q - Learning

В реальных задачах подход описанный выше не работает. Нам заранее не известны ни состояния ни вероятности переходов.




Q-Learning

мы не знаем вероятности переходов. Мы можем полько генерировать траектории и учиться на них.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-19.png" alt="Drawing" width="700px;"/>



<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-31.png" alt="Drawing" width="700px;"/>


Q - штрих рассчетное значение, Q без штриха то что было в таблице, альфа шаг обучения.

**temporal difference**

Пусть у нас есть оценка для текущего состояния 𝑆𝑡 и действия 𝐴𝑡 𝑄(𝑆𝑡,𝐴𝑡). Сделаем один шаг, получим реальную награду r, посмотрим оценку на следующем шаге 𝑚𝑎𝑥𝑄(𝑆𝑡+1,𝐴). Но в идеале 𝑟+𝑚𝑎𝑥𝑄(𝑆𝑡,𝐴)=𝑄(𝑆𝑡,𝐴𝑡), но если есть разница, то ее можно пытаться исправить.

Идея TD в том, чтобы смотреть на разницу между нашей прошлой оценкой и уточненной и использовать эту информацию

Temporal - так-как есть оценка сейчас и есть уточненная оценка через один шаг

В теории можно смотреть через n-шагов

### Deep Q-Learning

Есть задачи где количество состояний и переходов огромно и поддерживать таблицу значений не представляется возможным.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-6.png" width="700px;"/>

например, если мы учим алгоритм играть в простую компьютерную игру, состоянием будет изображение экрана (210x160).

В таких ситуациях Q- функцию не считают в явном виде, а аппроксимируют нейросетью.


**Possible architectures**

<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/yds-4-10.png" alt="Drawing" width="850px;"/>


    
    
Предпочтительнее, чтобы модель предсказывала Q -значения для всех действий.



### Loss



<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-11_.png" alt="Drawing" width="600px;"/>

<center><em> Approximate Q-learning <em><center> 

### Алгоритм обучения


<img src="https://edunet.kea.su/repo/src/L15_RL/img_license/yds-4-14.png" alt="Drawing" width="700px;"/>

<center><em> Basic deep Q-learning <em><center> 

$\epsilon -greedy$ нужна для исследования среды

### Experience replay


**Проблемма**:


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-16.png" alt="Drawing" width="700px;"/>

В большинстве окружений информация, получаемая агентом распределена не независимо. Т.е. последовательные наблюдения агента сильно коррелированы между собой (что понятно из интуитивных соображений, т.к. большинство окружений, в которых применяется RL, предполагают, что все изменения в них последовательны). Корреляция примеров ухудшает сходимость стохастического градиентного спуска. Таким образом нам нужен способ, который позволяет улучшить распределение примеров для обучения (устранить или снизить корреляцию между ними). 



<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-62.png" alt="Drawing" width="700px;"/>

Обычно используется метод **проигрывания опыта (experience replay)**. Суть этого метода в том, что мы сохраняем некоторое количество примеров (состояние, действия, вознаграждение) в специальном буфере и для обучения выбираем случайные мини-батчи из этого буфера.

Так же **experience replay** позволяет агенту эффективнее использовать свой прошлый опыт.


# Другие улучшения DQN

## Prioritized Experience Replay

Минибатчи из памяти выбираются не с равномерным распределением, а добавляем туда больше примеров, в которых предсказанные значения Q сильнее всего отличаются от корректных. Т.е. примеры с максимальным **TD error** получают максимальный приоритет.

## Dueling networks

Основная идея в том, что мы разделяем нашу сеть на две головы, одна из которых предсказывает абсолютное значение состояния \\( V(S) \\), а вторая - относительное преимущество одних действий над другими \\( A(s, a) = Q(s, a) - V(s) \\). Это преимущество называется advantage. Далее из этих двух значений мы собираем нашу Q-функцию, как \\( Q(s,a) = V(s) + A(a) \\)

## Noisy nets

Т.к. по мере обучения агент будет стремиться выбирать состояния с максимальным Q, среди уже исследованных, это может помешать ему найти более эффективные состояния, в которых его ещё не было. Одним из решений этой проблемы является использование детерминированной и случайной нейросети, распределение параметров которой так же обучается с помощью градиентного спуска.

## Multi-step learning/n-step learning

Основная идея в том, чтобы считать функцию ценности не по двум соседним примерам, а сразу по n. Это позволяет сети лучше запоминать длинные последовательности действий.

## Distributional RL

Детерминированное значение Q заменяется случайным распределением Z с некоторыми параметрами, которые определяются в ходе обучения.

# Rainbow

State of the art в развитии Q-обучения - набор перечисленных выше твиков. На графике ниже сравнение различных алгоритмов по количеству очков, усреднённое по играм Atari в сравнении со средними результатами человека.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/zap/rainbow_dqn.png" width="600">

## Gym - библиотека сред для RL


В комментариях к коду выше, говорилось о совместимости с Gym

*"Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball."*



form [openai](https://openai.com/projects/)
https://gym.openai.com/docs/


Список окружений:

https://gym.openai.com/envs/#classic_control


https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb

...

**Пример окружения** 
В этом примере загружается готовая среда: машинка должна заехать на горку. Функция награды встроенная. А агент - случайное воздействие. Оно ничему не учится, лишь хаотически выдаёт действия, но это показывает где взять все необходимые данные для обучения.

| Num        | Observation           | Min  | Max |
| ------------- |:-------------:|:-------------:|-------------:|
| 0      | position | -1.2 | 0.6 |
| 1      | velocity | -0.07 | 0.07 |

| Num        | Action |
| ------------- |-------------:|
| 0	| push left |
| 1	| no push |
| 2	| push right |

In [ ]:
# Install necessary package for show visualization in Colab
# https://stackoverflow.com/questions/50107530/how-to-render-openai-gym-in-google-colab

!apt-get install -y xvfb python-opengl
!pip install gym pyvirtualdisplay

In [ ]:
# Define some helper functions and init virtual display
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")

def draw(env):
    screen = env.render(mode='rgb_array')
    plt.imshow(screen)
    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())

from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

In [ ]:
query_environment("MountainCar-v0")

https://github.com/openai/gym/wiki/MountainCar-v0

In [ ]:
#import gym
env = gym.make("MountainCar-v0")
MAX_NUM_EPISODES = 1

for episode in range(MAX_NUM_EPISODES):
    done = False
    obs = env.reset()
    total_reward = 0.0 # To keep track of the total reward obtained in each episode
    step = 0
    while not done:
        draw(env)       
        action = env.action_space.sample()

        # Sample random action.
        # This will be replaced by our agent's action
        # when we # start developing the agent algorithms

        #action = int(input()) 0,1,2
        
        next_state, reward, done, info = env.step(action)
        # Send the action to the environment and receive       
        # the next_state, reward and whether done or not
        total_reward += reward
        step += 1
        obs = next_state
        
        if step > 50:
          # Forse exit in Colab demo
          done = True
          
    print("\n Episode #{} ended in {} steps. total_reward={}".format(episode, step+1,
total_reward))
env.close()

In [ ]:
wrapped_env = gym.wrappers.Monitor(env, 'test', force = True)
wrapped_env.reset()

for t in range(1000):
    action = env.action_space.sample()
    observation, reward, done, info = wrapped_env.step(action)
    if done:
        break

wrapped_env.close()

## Пример c CartPole DQN

Deep Q Network  = DQN 

Another env can be used without any modification of the code. State space should be a single vector, actions should be discrete.

CartPole is the simplest one. It should take several minutes to solve it.

For LunarLander it can take 1-2 hours to get 200 points (a good score) on Colab and training progress does not look informative.

In [ ]:
import os
################################################
# For CartPole
################################################

!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/atari_wrappers.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/utils.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/replay_buffer.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/framebuffer.py

!pip install gym[box2d]

!touch .setup_complete


# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import gym
ENV_NAME = 'CartPole-v1'

def make_env(seed=None):
    # some envs are wrapped with a time limit wrapper by default
    env = gym.make(ENV_NAME).unwrapped
    if seed is not None:
        env.seed(seed)
    return env

In [ ]:
import matplotlib.pyplot as plt
env = make_env()
env.reset()
plt.imshow(env.render("rgb_array"))
state_shape, n_actions = env.observation_space.shape, env.action_space.n

### Building a network

We now need to build a neural network that can map observations to state q-values.
The model does not have to be huge yet. 1-2 hidden layers with < 200 neurons and ReLU activation will probably be enough. Batch normalization and dropout can spoil everything here.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# those who have a GPU but feel unfair to use it can uncomment:
# device = torch.device('cpu')
device
print(state_shape)

In [ ]:
from torch import nn
class DQNAgent(nn.Module):
    def __init__(self, state_shape, n_actions, epsilon=0):

        super().__init__()
        self.epsilon = epsilon
        self.n_actions = n_actions
        self.state_shape = state_shape
        # Define your network body here. Please make sure agent is fully contained here
        assert len(state_shape) == 1
        state_dim = state_shape[0]
        
        
        # Define NN
        ##############################################
        hidden_size = 150
        self._nn = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
            nn.ReLU()
        )
        ##############################################

        
    def forward(self, state_t):
        """
        takes agent's observation (tensor), returns qvalues (tensor)
        :param state_t: a batch states, shape = [batch_size, *state_dim=4]
        """
        # Use your network to compute qvalues for given state
        
        ##############################################
        qvalues = self._nn(state_t)
        ##############################################

        assert qvalues.requires_grad, "qvalues must be a torch tensor with grad"
        assert (
            len(qvalues.shape) == 2 and 
            qvalues.shape[0] == state_t.shape[0] and 
            qvalues.shape[1] == n_actions
        )

        return qvalues

    def get_qvalues(self, states):
        """
        like forward, but works on numpy arrays, not tensors
        """
        model_device = next(self.parameters()).device
        states = torch.tensor(states, device=model_device, dtype=torch.float32)
        qvalues = self.forward(states)
        return qvalues.data.cpu().numpy()

    def sample_actions(self, qvalues):
        """pick actions given qvalues. Uses epsilon-greedy exploration strategy. """
        epsilon = self.epsilon
        batch_size, n_actions = qvalues.shape

        random_actions = np.random.choice(n_actions, size=batch_size)
        best_actions = qvalues.argmax(axis=-1)

        should_explore = np.random.choice(
            [0, 1], batch_size, p=[1-epsilon, epsilon])
        return np.where(should_explore, random_actions, best_actions)

In [ ]:
agent = DQNAgent(state_shape, n_actions, epsilon=0.5).to(device)

In [ ]:
def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """ Plays n_games full games. If greedy, picks actions as argmax(qvalues). Returns mean reward. """
    rewards = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        for _ in range(t_max):
            qvalues = agent.get_qvalues([s])
            action = qvalues.argmax(axis=-1)[0] if greedy else agent.sample_actions(qvalues)[0]
            s, r, done, _ = env.step(action)
            reward += r
            if done:
                break

        rewards.append(reward)
    return np.mean(rewards)

### Experience Replay Buffer and Target Networks

#### The interface is fairly simple:
* `exp_replay.add(obs, act, rw, next_obs, done)` - saves (s,a,r,s',done) tuple into the buffer
* `exp_replay.sample(batch_size)` - returns observations, actions, rewards, next_observations and is_done for `batch_size` random samples.
* `len(exp_replay)` - returns number of elements stored in replay buffer.

In [ ]:
from replay_buffer import ReplayBuffer

exp_replay = ReplayBuffer(2000)


target_network = DQNAgent(agent.state_shape, agent.n_actions, epsilon=0.5).to(device)
# This is how you can load weights from agent into target network
target_network.load_state_dict(agent.state_dict())

### TD-Loss

Compute Q-learning TD error:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

With Q-reference defined as

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Where
* $Q_{target}(s',a')$ denotes Q-value of next state and next action predicted by __target_network__
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor defined two cells above.

In [ ]:
def compute_td_loss(states, actions, rewards, next_states, is_done,
                    agent, target_network,
                    gamma=0.99,
                    check_shapes=False,
                    device=device):
    """ Compute td loss using torch operations only. Use the formulae above. """
    states = torch.tensor(states, device=device, dtype=torch.float32)    # shape: [batch_size, *state_shape]
    actions = torch.tensor(actions, device=device, dtype=torch.int64)    # shape: [batch_size]
    rewards = torch.tensor(rewards, device=device, dtype=torch.float32)  # shape: [batch_size]
    # shape: [batch_size, *state_shape]
    next_states = torch.tensor(next_states, device=device, dtype=torch.float)
    is_done = torch.tensor(
        is_done.astype('float32'),
        device=device,
        dtype=torch.float32,
    )  # shape: [batch_size]
    is_not_done = 1 - is_done

    # get q-values for all actions in current states
    predicted_qvalues = agent(states)  # shape: [batch_size, n_actions]

    # compute q-values for all actions in next states
    # with torch.no_grad():
    predicted_next_qvalues = target_network(next_states)  # shape: [batch_size, n_actions]
    
    # select q-values for chosen actions
    predicted_qvalues_for_actions = predicted_qvalues[range(len(actions)), actions]  # shape: [batch_size]

    # compute V*(next_states) using predicted next q-values
    ##############################################
    next_state_values = predicted_next_qvalues.max(axis=-1)[0]
    ##############################################

    assert next_state_values.dim() == 1 and next_state_values.shape[0] == states.shape[0], \
        "must predict one value per state"

    # compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
    # at the last state use the simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
    # you can multiply next state values by is_not_done to achieve this.
    ###############################################
    target_qvalues_for_actions = rewards + gamma * next_state_values * is_not_done
    ##############################################

    # mean squared error loss to minimize
    loss = torch.mean((predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)

    if check_shapes:
        assert predicted_next_qvalues.data.dim() == 2, \
            "make sure you predicted q-values for all actions in next state"
        assert next_state_values.data.dim() == 1, \
            "make sure you computed V(s') as maximum over just the actions axis and not all axes"
        assert target_qvalues_for_actions.data.dim() == 1, \
            "there's something wrong with target q-values, they must be a vector"

    return loss

### Main loop

In [ ]:
from tqdm import trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

In [ ]:
import random
import numpy as np

# your favourite random seed
seed = 2

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

env = make_env(seed)
state_dim = env.observation_space.shape
n_actions = env.action_space.n
state = env.reset()

agent = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network.load_state_dict(agent.state_dict())

Один агент для игры, второй для обучения. Переодически веса обученного агаента копируются в "игрока"

In [ ]:
g = list()
def play_and_record(initial_state, agent, env, exp_replay, n_steps=1):
    """
    Play the game for exactly n_steps, record every (s,a,r,s', done) to replay buffer. 
    Whenever game ends, add record with done=True and reset the game.
    It is guaranteed that env has done=False when passed to this function.

    PLEASE DO NOT RESET ENV UNLESS IT IS "DONE"

    :returns: return sum of rewards over time and the state in which the env stays
    
    hint: use agent.sample.actions
    """
    s = initial_state
    sum_rewards = 0

    # Play the game for n_steps as per instructions above
    for _ in range(n_steps):
        qvalues = agent.get_qvalues([s])
        
        action = agent.sample_actions(qvalues)[0]
        # action = action.argmax(axis=-1)[0]
        state, reward, done, _ = env.step(action)
        sum_rewards += reward
        g.append(reward)
        exp_replay.add(s, action, reward, state, done)
        
        if done:
            state = env.reset()
        
        s = state

    return sum_rewards, s

In [ ]:
# import src.L15_RL.lib.utils
import utils

REPLAY_BUFFER_SIZE = 10**4

exp_replay = ReplayBuffer(REPLAY_BUFFER_SIZE)
for i in range(100):
    if not utils.is_enough_ram(min_available_gb=0.1):
        print("""
            Less than 100 Mb RAM available. 
            Make sure the buffer size in not too huge.
            Also check, maybe other processes consume RAM heavily.
            """
             )
        break
    play_and_record(state, agent, env, exp_replay, n_steps=10**2)
    if len(exp_replay) == REPLAY_BUFFER_SIZE:
        break
print(len(exp_replay))

In [ ]:
timesteps_per_epoch = 1
batch_size = 32
total_steps = 4 * 10**4
decay_steps = 1 * 10**4

opt = torch.optim.Adam(agent.parameters(), lr=1e-4)

init_epsilon = 1
final_epsilon = 0.1

loss_freq = 20
refresh_target_network_freq = 100
eval_freq = 1000

max_grad_norm = 5000

mean_rw_history = []
td_loss_history = []
grad_norm_history = []
initial_state_v_history = []
step = 0

def wait_for_keyboard_interrupt():
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        pass

In [ ]:
state = env.reset()
with trange(step, total_steps + 1) as progress_bar:
    for step in progress_bar:
        if not utils.is_enough_ram():
            print('less that 100 Mb RAM available, freezing')
            print('make sure everything is ok and use KeyboardInterrupt to continue')
            wait_for_keyboard_interrupt()

        agent.epsilon = utils.linear_decay(init_epsilon, final_epsilon, step, decay_steps)

        # play
        _, state = play_and_record(state, agent, env, exp_replay, timesteps_per_epoch)

        # train
        # <YOUR CODE: sample batch_size of data from experience replay>
        s,a,r,next_s, is_done = exp_replay.sample(batch_size)
        # loss = <YOUR CODE: compute TD loss>
        loss = compute_td_loss(s, a, r, next_s, is_done, agent, target_network)

        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
        opt.step()
        opt.zero_grad()

        if step % loss_freq == 0:
            td_loss_history.append(loss.data.cpu().item())
            grad_norm_history.append(grad_norm)

        if step % refresh_target_network_freq == 0:
            # Load agent weights into target_network
            # <YOUR CODE>
            target_network.load_state_dict(agent.state_dict())

        if step % eval_freq == 0:
            mean_rw_history.append(evaluate(
                make_env(seed=step), agent, n_games=3, greedy=True, t_max=1000)
            )
            initial_state_q_values = agent.get_qvalues(
                [make_env(seed=step).reset()]
            )
            initial_state_v_history.append(np.max(initial_state_q_values))

            clear_output(True)
            print("buffer size = %i, epsilon = %.5f" %
                (len(exp_replay), agent.epsilon))

            plt.figure(figsize=[16, 9])

            plt.subplot(2, 2, 1)
            plt.title("Mean reward per episode")
            plt.plot(mean_rw_history)
            plt.grid()

            assert not np.isnan(td_loss_history[-1])
            plt.subplot(2, 2, 2)
            plt.title("TD loss history (smoothened)")
            plt.plot(utils.smoothen(td_loss_history))
            plt.grid()

            plt.subplot(2, 2, 3)
            plt.title("Initial state V")
            plt.plot(initial_state_v_history)
            plt.grid()

            plt.subplot(2, 2, 4)
            plt.title("Grad norm history (smoothened)")
            plt.plot(utils.smoothen(grad_norm_history))
            plt.grid()

            plt.show()

In [ ]:
final_score = evaluate(
  make_env(),
  agent, n_games=30, greedy=True, t_max=1000
)
print('final score:', final_score)
assert final_score > 300, 'not good enough for DQN'
print('Well done')

# Рекомендованная литература:

* Barto Sutton Reinforcement Learning: An Introduction (классика теории)
* Lapan Deep Reinforcement Learning Hands-On Second Edition (Книга с большим количеством примеров по RL)
* Sudharsan Ravichandiran Hands-On Reinforcement Learning with Python (Тоже много примеров)

Прочая дополнительная литература:

* Alex Zai and Brandon Brown Deep Reinforcement Learning in Action


Полезные ссылки:
* https://www.alexirpan.com/2018/02/14/rl-hard.html Deep Reinforcement Learning Doesn't Work Yet
* https://openai.com/blog/faulty-reward-functions/  Faulty Reward Functions in the Wild

== Реализации ==
* [BURLAP](http://burlap.cs.brown.edu) (Brown-UMBC Reinforcement Learning and Planning) — библиотека одно- и многоагентных алгоритмов планирования и обучения с подкреплением, язык Java, лицензия LGPL
* [MMLF](http://mmlf.sourceforge.net/) (Maja Machine Learning Framework) — библиотека алгоритмов обучения с подкреплением и набор тестовых сред для их проверки, язык Python, лицензия GPL
* [OpenAI Gym](https://gym.openai.com) — платформа для разработки и сравнения алгоритмов обучения с подкреплением от [[OpenAI]], язык Python, лицензия MIT
* [PyBrain](http://www.pybrain.org/) — библиотека алгоритмов машинного обучения, язык Python, лицензия BSD
* [RLPy](https://rlpy.readthedocs.io/en/latest)— библиотека для проведения экспериментов по обучению с подкреплением, язык Python, 3-х пунктовая лицензия BSD
* [Teachingbox](http://servicerobotik.hs-weingarten.de/en/teachingbox.php) — инструментарий для разработки алгоритмов обучения с подкреплением, язык Java, лицензия GPL